In [1]:
#起手式
import numpy as np 
import math
import os
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import timm
from torchvision import transforms
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, LinearLR
from torch.optim.lr_scheduler import SequentialLR

from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from PIL import Image

#随机种子
def set_seed(seed):
    seed = int(seed)
    if seed < 0 or seed > (2**32 - 1):
        raise ValueError("Seed must be between 0 and 2**32 - 1")
    else:
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
set_seed(16)

#当前设备
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'使用{device}训练')

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

使用cuda训练


In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.Normalize(mean=(0.485,0.456,0.406),
                         std=(0.229,0.224,0.225)),
])
class Dataset():
    def __init__(self, df, transform, scaler_target):
        self.samples = list(df.itertuples(index=False))
        self.image_dir = "/kaggle/input/csiro-biomass"
        self.targets = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
        self.transform = transform
        self.scaler_target = scaler_target

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        row = self.samples[idx]

        img_path = os.path.join(self.image_dir, row.image_path)
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)

        target = torch.tensor([getattr(row, t) for t in self.targets], dtype=torch.float32)
        target = torch.tensor(self.scaler_target.transform([target.numpy()])[0], dtype=torch.float32)

        return image, target

In [3]:
#读取文件 并统计图片对应标签值
df = pd.read_csv("/kaggle/input/csiro-biomass/train.csv")
df = df[["image_path", "target_name", "target"]]
df = df.pivot_table(index=["image_path"],columns='target_name',values="target").reset_index()
print('文件读取成功',df.head(5))

#转换结构 列为target_name 值为target

#df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

# 目标值缩放 避免梯度爆炸
scaler_target = StandardScaler()
targets=['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
scaler_target.fit(df[targets].values)

文件读取成功 target_name              image_path  Dry_Clover_g  Dry_Dead_g  Dry_Green_g  \
0            train/ID1011485656.jpg        0.0000     31.9984      16.2751   
1            train/ID1012260530.jpg        0.0000      0.0000       7.6000   
2            train/ID1025234388.jpg        6.0500      0.0000       0.0000   
3            train/ID1028611175.jpg        0.0000     30.9703      24.2376   
4            train/ID1035947949.jpg        0.4343     23.2239      10.5261   

target_name  Dry_Total_g    GDM_g  
0                48.2735  16.2750  
1                 7.6000   7.6000  
2                 6.0500   6.0500  
3                55.2079  24.2376  
4                34.1844  10.9605  


StandardScaler()

In [4]:
# # !pip install -U -q keras
# # ! pip install -U -q git+https://github.com/keras-team/keras-hub.git
# import os
# os.environ["KERAS_BACKEND"] = "jax"  
# import keras_hub

# from keras_hub.layers import ViTImageConverter
# from keras_hub.models import ViTImageClassifierPreprocessor

In [5]:

from tensorflow import keras
#模型
class ViTBase16(nn.Module):
    def __init__(self):

        super(ViTBase16, self).__init__()
        # backbone = keras_hub.models.Backbone.from_preset("vit_base_patch16_224_imagenet")
        # preprocessor = keras_hub.models.ViTImageClassifierPreprocessor.from_preset("vit_base_patch16_224_imagenet")
        # self.model = keras_hub.models.ViTImageClassifier(
        #     backbone=backbone,
        #     num_classes=5,
        #     preprocessor=preprocessor,
        # )
        
        # # 加载权重
        self.model = timm.create_model('vit_base_patch16_224', pretrained=False)
        self.model.head = nn.Linear(self.model.head.in_features, 5)
        state_dict = torch.load('/kaggle/input/vision-transformer-vit-tutorial-baseline/model_5e_20201202-1908.pth')
        state_dict = {k.replace('model.', ''): v for k, v in state_dict.items()}
        self.model.load_state_dict(state_dict)
        

    def forward(self, x):
        x = self.model(x)
        return x


2026-01-04 16:18:15.210213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767543495.442847      25 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767543495.519839      25 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767543496.086616      25 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767543496.086663      25 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767543496.086666      25 computation_placer.cc:177] computation placer alr

In [6]:
# model = timm.create_model('vit_base_patch16_224', pretrained=False)
# model.head = nn.Linear(model.head.in_features, 5)
# state_dict = torch.load('/kaggle/input/vision-transformer-vit-tutorial-baseline/model_5e_20201202-1908.pth')
# # 去除 'model.' 前缀
# new_state_dict = {k.replace('model.', ''): v for k, v in state_dict.items()}

# # 加载权重
# model.load_state_dict(new_state_dict)

# model.parameters

In [7]:
class Full():
    def __init__(self,df,scaler_target,loss_fn,optimizer):
        #super().__init__()
        self.df = df
        self.scaler_target = scaler_target
        #self.model=model
        # self.loader=loader
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        
    def train(self,model,train_dataloader):
        #混合精度
        from torch.amp import autocast, GradScaler 
        scaler = GradScaler()
        print("启用混合精度训练")
        model.train()
        loss_sum = 0.0
    
        for x , Y in train_dataloader:
            x , Y= x.to(device) , Y.to(device)
    
            self.optimizer.zero_grad()
            with autocast(device):
                pred = model(x)
                loss = self.loss_fn(pred, Y)
                #print(loss)
            scaler.scale(loss).backward() 
            scaler.step(self.optimizer)
            scaler.update()
            
            loss_sum += loss.item()
    
        return loss_sum / len(train_dataloader)
        
    def test(self,model,test_dataloader):
        model.eval()
        loss_sum = 0.0
    
        with torch.no_grad():
            for x , Y in test_dataloader:
                x , Y= x.to(device),Y.to(device)
                
                pred = model(x)
                loss = self.loss_fn(pred, Y)
                loss_sum += loss.item()
    
        return loss_sum / len(test_dataloader)
        
    def res(self,best_model):
        df_test = pd.read_csv('/kaggle/input/csiro-biomass/test.csv')
        test_path = df_test["image_path"].values.tolist()
        test_target = df_test["target_name"].values.tolist()
        
        res = []
        with torch.no_grad():
            for img_path,target in zip(test_path,test_target):
                img_path = os.path.join('/kaggle/input/csiro-biomass', img_path)
                image = Image.open(img_path).convert("RGB")
                image = transform(image).to(device).unsqueeze(0)
        
                preds = best_model(image)
                preds = self.scaler_target.inverse_transform(preds.detach().cpu())
                preds=preds.reshape(-1)
                if target == 'Dry_Clover_g':
                    res.append(preds[0])
                elif target == 'Dry_Dead_g':
                    res.append(preds[1])
                elif target == 'Dry_Green_g':
                    res.append(preds[2])
                elif target == 'Dry_Total_g':
                    res.append(preds[3])
                else: #GDM_g
                    res.append(preds[4])
        
        #输出
        submission = pd.DataFrame({'sample_id':df_test['sample_id'],'target':res})
        submission.to_csv('submission.csv', index = False)
        print(submission)
        
  

In [8]:
def dataloader(flod):
    batch_size = 64
    
    train_dataset = Dataset(df[df['fold']!=flod], transform, scaler_target)
    test_dataset = Dataset(df[df['fold']==flod], transform, scaler_target)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=2)
    test_dataloader = DataLoader(test_dataset,batch_size=batch_size,num_workers=2)
    
    return train_dataloader,test_dataloader

In [9]:
def train_flod(df,scaler_target,train_dataloader,test_dataloader):
    #释放内存
    import gc
    
    torch.cuda.empty_cache()
    gc.collect()
    
    #定义模型
    model=ViTBase16()
    
    #多gpu运行
    model = model.to(device)
    if torch.cuda.device_count() > 1:
        print(f"使用 {torch.cuda.device_count()} 个GPU")
        model = torch.nn.DataParallel(model)
    else:
        model.to(device)
    
    #损失函数
    loss_fn = nn.MSELoss()
    epoch = 10
    
    optimizer = optim.AdamW(model.parameters(),
                          lr=1e-5,              
                          weight_decay=5e-4
                         )
    
    min_loss = np.inf
    best_model = model #最优模型
    
    train_loss_list=[]
    test_loss_list=[]
    lr_list=[]
    
    #耐心
    # n= 10
    # if n < int(epoch*0.3): n= int(epoch*0.3)
    # print("早停耐心：",n)

    #训练
    full=Full(df,scaler_target,loss_fn,optimizer)
    
    for i in range(epoch):
        train_loss=full.train(model,train_dataloader)
        test_loss=full.test(model,test_dataloader)
        train_loss_list.append(train_loss)
        test_loss_list.append(test_loss)
        
        print(f"第{i+1}/{epoch}迭代 | 训练损失:{train_loss} | 测试损失值:{test_loss}")

        
        if min_loss >= test_loss:
            min_loss = test_loss
            best_model=model

    full.res(best_model)
    torch.save(model.state_dict(), 'VIT—CSIRO')
    print("保存模型：VIT—CSIRO")


In [10]:
from sklearn.model_selection import KFold, GroupKFold, StratifiedGroupKFold
NFOLD = 5
kfold = KFold(n_splits=NFOLD, shuffle=True, random_state=42)

df['fold'] = None
for i, (trn_idx, val_idx) in enumerate(kfold.split(df.index)):
    df.loc[val_idx, 'fold'] = i
df

target_name,image_path,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g,fold
0,train/ID1011485656.jpg,0.0000,31.9984,16.2751,48.2735,16.2750,3
1,train/ID1012260530.jpg,0.0000,0.0000,7.6000,7.6000,7.6000,4
2,train/ID1025234388.jpg,6.0500,0.0000,0.0000,6.0500,6.0500,2
3,train/ID1028611175.jpg,0.0000,30.9703,24.2376,55.2079,24.2376,0
4,train/ID1035947949.jpg,0.4343,23.2239,10.5261,34.1844,10.9605,3
...,...,...,...,...,...,...,...
352,train/ID975115267.jpg,40.0300,0.0000,0.8000,40.8300,40.8300,1
353,train/ID978026131.jpg,24.6445,4.1948,12.0601,40.8994,36.7046,0
354,train/ID980538882.jpg,0.0000,1.1457,91.6543,92.8000,91.6543,4
355,train/ID980878870.jpg,32.3575,0.0000,2.0325,34.3900,34.3900,0


In [11]:
for i in range(NFOLD):

    train_dataloader,test_dataloader = dataloader(flod=i)
    train_flod(df,scaler_target,train_dataloader,test_dataloader)
    

使用 2 个GPU
启用混合精度训练
第1/10迭代 | 训练损失:1.8722216367721558 | 测试损失值:1.145126074552536
启用混合精度训练
第2/10迭代 | 训练损失:1.1908068418502809 | 测试损失值:0.8428778052330017
启用混合精度训练
第3/10迭代 | 训练损失:0.9533923983573913 | 测试损失值:0.7336410880088806
启用混合精度训练
第4/10迭代 | 训练损失:0.7859128832817077 | 测试损失值:0.6164080798625946
启用混合精度训练
第5/10迭代 | 训练损失:0.7276534914970398 | 测试损失值:0.5722047984600067
启用混合精度训练
第6/10迭代 | 训练损失:0.6581467092037201 | 测试损失值:0.5323698371648788
启用混合精度训练
第7/10迭代 | 训练损失:0.5892581284046173 | 测试损失值:0.49404698610305786
启用混合精度训练
第8/10迭代 | 训练损失:0.5337244689464569 | 测试损失值:0.48400601744651794
启用混合精度训练
第9/10迭代 | 训练损失:0.45971248745918275 | 测试损失值:0.473890095949173
启用混合精度训练
第10/10迭代 | 训练损失:0.4286569356918335 | 测试损失值:0.4252179488539696
                    sample_id     target
0  ID1001187975__Dry_Clover_g  -3.091680
1    ID1001187975__Dry_Dead_g  23.414399
2   ID1001187975__Dry_Green_g  43.476594
3   ID1001187975__Dry_Total_g  64.109573
4         ID1001187975__GDM_g  30.839498
保存模型：VIT—CSIRO
使用 2 个GPU
启用混合精度训练
第1/10迭代 